# What about the LLMs?

LLMs are reputed to have revolutionised automatic language processing. Since the introduction of BERT-type models, all language processing applications have been based on LLMs, of varying degrees of sophistication and size. These models are trained on multiple tasks and are therefore capable of performing new tasks without learning, simply from a prompt. This is known as "zero-shot learning" because there is no learning phase as such. We are going to test these models on our classification task.

Huggingface is a Franco-American company that develops tools for building applications based on Deep Learning. In particular, it hosts the huggingface.co portal, which contains numerous Deep Learning models. These models can be used very easily thanks to the [Transformer] library (https://huggingface.co/docs/transformers/quicktour) developed by HuggingFace.

Using a transform model in zero-shot learning with HuggingFace is very simple: [see documentation](https://huggingface.co/tasks/zero-shot-classification)

However, you need to choose a suitable model from the list of models compatible with Zero-Shot classification. HuggingFace offers [numerous models](https://huggingface.co/models?pipeline_tag=zero-shot-classification). 

The classes proposed to the model must also provide sufficient semantic information for the model to understand them.

**Question**:

* Write a code to classify an example of text from an article in Le Monde using a model transformed using zero-shot learning with the HuggingFace library.
* choose a model and explain your choice
* choose a formulation for the classes to be predicted
* show that the model predicts a class for the text of the article (correct or incorrect, analyse the results)
* evaluate the performance of your model on 100 articles (a test set).
* note model sizes, processing times and classification results


Notes :
* make sure that you use the correct Tokenizer when using a model 
* start testing with a small number of articles and the first 100's of characters for faster experiments.

In [63]:
# Choisir modèle multilingue, essayer avec un petit modèle et avec un grand.

In [64]:
import pandas as pd

In [65]:
 articles_df = pd.read_csv('LeMonde2003_9classes.csv.gz')

In [66]:
# Filter out the UNE class
articles_df = articles_df[articles_df['category'] != 'UNE']

In [67]:
sample = articles_df.sample(100)

In [68]:
sample

,text,category
4053,le responsable des services de sécurité égypti...,INT
6947,les groupes américain et allemand associés au ...,ENT
26813,les syndicats de police se sont succédé au min...,SOC
21247,loto résultats des tirages no 64 effectués sam...,SPO
773,nouvelle étape dans la relève de génération à ...,INT
...,...,...
13277,l'armée républicaine irlandaise ira principale...,INT
13391,les organisations syndicales ont rejeté lundi ...,FRA
7217,fantasque accidentel ou grand obsessionnel du ...,ART
25531,le principal actionnaire du groupe britannique...,ENT


In [69]:
texts = sample.text

In [70]:
articles_df.category.unique()

array(['SPO', 'ART', 'FRA', 'SOC', 'INT', 'ENT'], dtype=object)

In [71]:
candidate_labels = ['companies', 'international', 'arts', 'society', 'France', 'sports', 'books', 'television']

# Dictionnaire de correspondance
corresp_dict = {
    'companies':'ENT',
    'international':'INT',
    'arts':'ART',
    'society':'SOC',
    'France':'FRA',
    'sports':'SPO',
    'books':'LIV'
}

In [72]:
import torch
import tensorflow as tf
from transformers import pipeline

In [73]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [75]:
predictions = []
scores = []

# Mesure du temps de traitement
start_time = time.time()

for text in texts.to_list():
    result = classifier(text[:100], candidate_labels, multi_label=True)
    
    # Trouver l'indice du score le plus élevé
    max_score_index = result['scores'].index(max(result['scores']))

    # Sélectionner le label correspondant et son score
    selected_label = result['labels'][max_score_index]
    selected_score = result['scores'][max_score_index]

    # Ajouter les éléments aux listes
    predictions.append(selected_label)
    scores.append(selected_score)

    predictions.append(output)
    
# Temps de traitement
end_time = time.time()
processing_time = end_time - start_time

# Taille du modèle
model_dir = os.path.join(os.path.expanduser('~'), '.cache', 'huggingface', 'transformers', 'facebook', 'bart-large-mnli')
model_size = sum(os.path.getsize(os.path.join(model_dir, f)) for f in os.listdir(model_dir) if os.path.isfile(os.path.join(model_dir, f)))

# Afficher les résultats
print(f"Time processing: {processing_time:.4f} seconds")
print(f"Model size: {model_size / (1024**2):.2f} MB")

sample['prediction'] = predictions
sample['scores'] = scores

NameError: name 'classifier' is not defined

In [76]:
# Remplacer les valeurs dans la colonne en utilisant le dictionnaire
sample['prediction'] = sample['prediction'].replace(corresp_dict)

KeyError: 'prediction'

In [77]:
# Calculer le nombre de classifications correctes
count_equal = (sample['category'] == sample['prediction']).sum()
total_lines = len(sample)
accuracy_percentage = (count_equal / total_lines) * 100

KeyError: 'prediction'

In [78]:
print(f"L'évaluation des performances du modèle montre que 'category' et 'prediction' sont égales dans {count_equal} cas sur {total_lines} lignes, soit un taux de précision de {accuracy_percentage:.2f}%")

NameError: name 'count_equal' is not defined